In [87]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

In [88]:
!ls ../../nf_data/nf_wf_fg_singlecell

atac_fastqs_files_local.csv  rna_fastqs_files_vm-cellatlas.csv
atac_fastqs_files_vm.csv     rna_fastqs_files_vm-dogma-dig.csv
na.na			     rna_fastqs_files_vm-kb-python.csv
rna_fastqs_files_local.csv   specs


In [89]:
!gcloud auth list
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [90]:
import os
# os.environ["TOWER_ACCESS_TOKEN"] = 'XXX'

# !echo $TOWER_ACCESS_TOKEN

In [91]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [92]:
import os
os.getcwd()

'/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna'

In [93]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [94]:
# !ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell

In [122]:
%%writefile nextflow_vm.config

includeConfig 'conf/conda.config'


manifest {
  description = 'A nexflow pipeline for fg single cell' 
  mainScript = 'nf_wf_fg_singlecell_rna.nf'
}

profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  docker_local {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work/RNA'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }
    
}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.ORGANISM = 'human'
params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv'
params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()
params.SYNAPSE_CONFIG = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/synapse.config'
params.SYNAPSE_IDS = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/synapse_ids.csv'
params.SUBPOOL = 'none'
params.UMI_CUTOFF=2
params.GENE_CUTOFF=3


Overwriting nextflow_vm.config


In [123]:
!cat /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv

R1_fastq_gz	R2_fastq_gz	R3_fastq_gz	spec	whitelist
/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/fastqs/RNA/SRR18677638_1.fastq.gz	/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/fastqs/RNA/SRR18677638_2.fastq.gz	/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na.na	/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/nf_spec_dogmaseq_dig.yaml	/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/RNA-737K-arc-v1.txt

In [124]:
# !head /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/nf_spec_dogmaseq_dig.yaml

In [125]:
# !head /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/RNA-737K-arc-v1.txt

In [126]:
# !ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna


In [127]:
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/work


# !gsutil rm -r gs://eila-nextflow-bucket/work/RNA




In [128]:
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile  -with-tower

# docker_local



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile docker_profile

In [129]:
# !cat .nextflow.log

In [130]:
# import datetime
# now = datetime.datetime.now()
# print(now)

In [ ]:

!nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile  -with-tower -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local -resume





# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile google_batch_profile -resume

N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_rna.nf` [astonishing_boyd] DSL2 - revision: 5d86a969b7
Nov 10, 2023 11:01:04 PM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
null
/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv
[-        ] process > run_seqspec_modify_rna     -
[-        ] process > run_seqspec_print          -
[-        ] process > run_seqspec_index_rna_kb   -
[-        ] process > run_kb_ref                 -
[-        ] process > run_kb_count               -
[-        ] process > scrna_calculate_qc_metrics -
after run_seqspec_modify_rna
after run_seqspec_print
after run_seqspec_index_rna_kb
technology is $technology
after genome_fasta_ch
after genome_gtf_ch
after run_kb_ref
after run_kb_count

Monitor the execution with Nextflow Tower using this URL: https://tower.nf/user/eila/watch/3EQgb5s4sRI3QD
[-        ] process > run_seqspec_modify_rna 

In [ ]:
!cat .nextflow.log 

In [ ]:
import datetime
now = datetime.datetime.now()
print(now)